In [ ]:
import os
from os.path import join, basename
from glob import glob
import rasterio as rio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np
import sys
import rioxarray as rxa
sys.path.insert(0, '/Users/zachkeskinen/Documents/testing/uavsar_pytools/')
from uavsar_pytools import UavsarImage, UavsarScene, UavsarCollection

In [ ]:
img_url = 'http://uavsar.asfdaac.alaska.edu/UA_alamos_35915_20008-000_20013-000_0007d_s01_L090_02/alamos_35915_20008-000_20013-000_0007d_s01_L090HH_02.amp1.grd'
work_dir = '/tmp/pytools_test/'
image = UavsarImage(url = img_url, work_dir=work_dir)
image.url_to_tiff()

In [ ]:
with rio.open(image.out_fp) as src:
    arr = src.read(1)
    vmin, vmax = np.nanquantile(arr, [0.1,0.9])
    show(src, vmin = vmin, vmax = vmax)

In [ ]:
scene_url = 'https://datapool.asf.alaska.edu/INTERFEROMETRY_GRD/UA/Slumgl_12502_13086-002_13090-002_0007d_s01_L090_01_int_grd.zip'
scene_url = 'https://datapool.asf.alaska.edu/INTERFEROMETRY_GRD/UA/stlake_09127_20002-009_20007-005_0013d_s01_L090_01_int_grd.zip'
work_dir = '/tmp/pytools_test/'
scene = UavsarScene(url = scene_url, work_dir=work_dir, clean = True)
scene.url_to_tiffs()

In [ ]:
plt.rcParams['axes.facecolor']='lightgrey'
f, axes = plt.subplots(3,1, figsize = (12,8), dpi = 160)
ax1, ax2, ax3 = axes
mag = np.abs(scene.images['int']['array'])
phase = np.angle(scene.images['int']['array'])

ax1.imshow(scene.images['cor']['array'])
ax1.set_title('Coherence')
ax2.imshow(mag, vmin = 0, vmax = 1)
ax2.set_title('Amplitude')
ax3.imshow(phase, vmin = -2, vmax = 2)
ax3.set_title('Phase')
for ax in axes:
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
plt.tight_layout()

In [ ]:
collection_name = 'Grand Mesa, CO'
work_dir = '/tmp/pytools_test/'
collection = UavsarCollection(collection = collection_name, work_dir = work_dir, inc = True, dates = ('2020-03-01','2020-04-02'))
collection.find_urls()
print(len(collection.results))
collection.results_to_tiffs()

In [ ]:
from uavsar_pytools.snow_depth_inversion import depth_from_phase, phase_from_depth
from uavsar_pytools.georeference import reproject_clip_mask

In [ ]:
work_dir = '/tmp/pytools_test/'
unw_fp = glob(join('/tmp/pytools_test/', 'grmesa_27416_20013-004_20017-006_0015d_s01_L090_01_int_grd','*VV*.unw.grd.tiff'))[0]
inc_fp = glob(join('/tmp/pytools_test/', 'grmesa_27416_20013-004_20017-006_0015d_s01_L090_01_int_grd','*inc.tiff'))[0]
new_inc_fp = join(work_dir, 'inc_resampled.tif')

reproject_clip_mask(inc_fp, unw_fp, out_fp=new_inc_fp)

In [ ]:
f, ax = plt.subplots()
rxa.open_rasterio(new_inc_fp).plot(ax = ax, add_colorbar=False)
plt.title('Incidence Angle')

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
plt.show()
f, ax = plt.subplots()
rxa.open_rasterio(unw_fp).plot(ax = ax, add_colorbar=False)
plt.title('Phase Change')

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
plt.show()

In [ ]:
f, ax = plt.subplots()
rxa.open_rasterio(unw_fp).plot(ax = ax, add_colorbar=False, vmin = -2, vmax = 2,cmap = 'magma')
plt.title('Phase Change')

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
with rio.open(unw_fp) as unw_src, rio.open(new_inc_fp) as inc_src:
    unw = unw_src.read(1)
    inc = inc_src.read(1)
    dz = depth_from_phase(unw, inc, permittivity=1.2)
    dphase = phase_from_depth(dz, inc, permittivity=1.2)
f, (ax1, ax2, ax3) = plt.subplots(1,3,figsize = (12,8))
ax1.imshow(unw, vmin = -np.pi, vmax= np.pi)
ax2.imshow(dz, vmin = -0.5, vmax = 0.5)
phase_diff = unw - dphase
ax3.imshow(phase_diff, vmin = -0.1, vmax= 0.1)
np.nanmean(phase_diff)

In [ ]:
f, ax1 = plt.subplots(1,1, figsize = (12,8), dpi = 160)

ax1.imshow(dz, vmin = 0, vmax = 0.3, interpolation='nearest')
ax1.set_title('Snow Depth Change')
ax1.xaxis.set_visible(False)
ax1.yaxis.set_visible(False)

In [ ]:
polsar_url = 'https://datapool.asf.alaska.edu/PROJECTED/UA/eelriv_24500_21059_002_211018_L090_CX_01_grd.zip'
scene = UavsarScene(url =polsar_url, work_dir=work_dir)
scene.download()
scene.unzip()

In [ ]:
from uavsar_pytools.polsar import H_A_alpha_decomp
from os.path import join

In [ ]:
work_dir = '/tmp/pytools_test/'
polsar_dir = join(work_dir, 'tmp', 'eelriv_24500_21059_002_211018_L090_CX_01_grd','bin_imgs')
H_A_alpha_decomp(polsar_dir, out_dir = join(work_dir, 'polsar_out'), parralel = True)

In [ ]:
polsar_url = 'https://datapool.asf.alaska.edu/PROJECTED/UA/eelriv_24500_21059_002_211018_L090_CX_01_grd.zip'
scene = UavsarScene(url =polsar_url, work_dir=work_dir)
scene.download()
scene.unzip()

In [ ]:
from uavsar_pytools.georeference import geolocate_uavsar
from uavsar_pytools.incidence_angle import calc_inc_angle

In [ ]:
!wget https://downloaduav2.jpl.nasa.gov/Release20/grmesa_26006_01/grmesa_26006_01_BC.dop
!wget https://downloaduav2.jpl.nasa.gov/Release20/grmesa_26006_01/grmesa_26006_01_BC_s1_2x8.llh
!wget https://downloaduav2.jpl.nasa.gov/Release20/grmesa_26006_01/grmesa_26006_01_BC_s1_2x8.lkv
!wget https://downloaduav2.jpl.nasa.gov/Release20/grmesa_26006_01/grmesa_26006_17002_001_170206_L090HH_01_BC.ann

In [ ]:
work_dir = '/tmp/pytools_test/'
lkv_fp = glob(join('/tmp/pytools_test/georef_inc_test', '*.lkv'))[0]
llh_fp = glob(join('/tmp/pytools_test/georef_inc_test', '*.llh'))[0]
ann_fp = glob(join('/tmp/pytools_test/georef_inc_test', '*.ann'))[0]
geolocate_uavsar(lkv_fp, ann_fp, out_dir = join(work_dir, 'geolocated_test'), llh_fp= llh_fp)

In [ ]:
work_dir = '/tmp/pytools_test/'
lkv_fp = glob(join('/tmp/pytools_test/georef_inc_test', '*.lkv'))[0]
llh_fp = glob(join('/tmp/pytools_test/georef_inc_test', '*.llh'))[0]
ann_fp = glob(join('/tmp/pytools_test/georef_inc_test', '*.ann'))[0]
geolocate_uavsar(llh_fp, ann_fp, out_dir = join(work_dir, 'geolocated_test'), llh_fp= llh_fp)